In [1]:
from google.colab import files
import pandas as pd

# Upload the file from your local machine
uploaded = files.upload()


# Load the dataset into a pandas DataFrame (Assume 'train.csv' is the file name)
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Display the first few rows
train_data.head()


Saving gender_submission.csv to gender_submission (1).csv
Saving test.csv to test (1).csv
Saving train.csv to train (1).csv


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
from sklearn.impute import SimpleImputer


train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch']

train_data.drop(['SibSp', 'Parch'], axis=1, inplace=True)


train_data['Title'] = train_data['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())


rare_titles = ['Lady', 'Countess', 'Sir', 'Jonkheer', 'Dona', 'Rev']
train_data['Title'] = train_data['Title'].apply(lambda x: 'Rare' if x in rare_titles else x)

train_data['Embarked'] = train_data['Embarked'].astype(str)

age_imputer = SimpleImputer(strategy='mean')
train_data['Age'] = age_imputer.fit_transform(train_data[['Age']])

embarked_imputer = SimpleImputer(strategy='most_frequent')
train_data['Embarked'] = embarked_imputer.fit_transform(train_data[['Embarked']]).ravel()

fare_imputer = SimpleImputer(strategy='mean')
train_data['Fare'] = fare_imputer.fit_transform(train_data[['Fare']])

train_data['Sex'] = train_data['Sex'].map({'male': 0, 'female': 1})

train_data = pd.get_dummies(train_data, columns=['Embarked'], drop_first=True)

train_data.drop(columns=['Name', 'Ticket', 'Cabin'], inplace=True, errors='ignore')

# Show first few rows of processed data
print(train_data.head())

   PassengerId  Survived  Pclass  Sex   Age     Fare  FamilySize Title  \
0            1         0       3    0  22.0   7.2500           1    Mr   
1            2         1       1    1  38.0  71.2833           1   Mrs   
2            3         1       3    1  26.0   7.9250           0  Miss   
3            4         1       1    1  35.0  53.1000           1   Mrs   
4            5         0       3    0  35.0   8.0500           0    Mr   

   Embarked_Q  Embarked_S  Embarked_nan  
0       False        True         False  
1       False       False         False  
2       False        True         False  
3       False        True         False  
4       False        True         False  


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_data['Title'] = label_encoder.fit_transform(train_data['Title'])

X = train_data.drop('Survived', axis=1)
y = train_data['Survived']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)


In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=42)

param_grid = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

grid_search.fit(X_train_scaled, y_train)

print("Best Hyperparameters: ", grid_search.best_params_)

best_rf_model = grid_search.best_estimator_


Fitting 5 folds for each of 324 candidates, totalling 1620 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
205 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/uti

Best Hyperparameters:  {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 150}


In [5]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(best_rf_model, X_train_scaled, y_train, cv=5)

print(f"Cross-validation Mean Accuracy: {cv_scores.mean() * 100:.2f}%")
print(f"Cross-validation Std Accuracy: {cv_scores.std() * 100:.2f}%")


Cross-validation Mean Accuracy: 82.86%
Cross-validation Std Accuracy: 1.81%


In [6]:
y_pred = best_rf_model.predict(X_val_scaled)

from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

print(classification_report(y_val, y_pred))


Validation Accuracy: 81.01%
              precision    recall  f1-score   support

           0       0.81      0.89      0.85       105
           1       0.81      0.70      0.75        74

    accuracy                           0.81       179
   macro avg       0.81      0.79      0.80       179
weighted avg       0.81      0.81      0.81       179



In [7]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X_train_scaled, y_train)

y_pred_xgb = xgb_model.predict(X_val_scaled)

accuracy_xgb = accuracy_score(y_val, y_pred_xgb)
print(f'Validation Accuracy (XGBoost): {accuracy_xgb * 100:.2f}%')

print(classification_report(y_val, y_pred_xgb))


Validation Accuracy (XGBoost): 79.89%
              precision    recall  f1-score   support

           0       0.82      0.84      0.83       105
           1       0.76      0.74      0.75        74

    accuracy                           0.80       179
   macro avg       0.79      0.79      0.79       179
weighted avg       0.80      0.80      0.80       179



In [8]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

imputer = SimpleImputer(strategy='mean')
imputer_fare = SimpleImputer(strategy='mean')

imputer.fit(train_data[['Age']])
imputer_fare.fit(train_data[['Fare']])

test_data['Age'] = imputer.transform(test_data[['Age']])
test_data['Fare'] = imputer_fare.transform(test_data[['Fare']])

if 'SibSp' in test_data.columns and 'Parch' in test_data.columns:
    test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch']
    test_data.drop(columns=[col for col in ['SibSp', 'Parch'] if col in test_data.columns], inplace=True)

if 'Name' in test_data.columns:
    test_data['Title'] = test_data['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())

rare_titles = ['Lady', 'Countess', 'Sir', 'Jonkheer', 'Dona', 'Rev']
test_data['Title'] = test_data['Title'].apply(lambda x: 'Rare' if x in rare_titles else x)

test_data = pd.get_dummies(test_data, columns=['Title'], drop_first=True)

test_data['Sex'] = test_data['Sex'].map({'male': 0, 'female': 1})

test_data = pd.get_dummies(test_data, columns=['Embarked'], drop_first=True)

test_data.drop(columns=[col for col in ['Name', 'Ticket', 'Cabin'] if col in test_data.columns], inplace=True)

X_test = test_data.drop('PassengerId', axis=1)

for col in X_train.columns:
    if col not in X_test.columns:
        X_test[col] = 0

X_test = X_test[X_train.columns]

X_test_scaled = scaler.transform(X_test)

test_predictions = best_rf_model.predict(X_test_scaled)

submission = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': test_predictions})

submission.to_csv('titanic_predictions.csv', index=False)

submission.head()


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [9]:
from google.colab import files
files.download('titanic_predictions.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>